# Análisis de Valores Extremos (Outliers)

2 maneras para realizar dicho análisis:

* Criterio de una sola variable (la manera más clásica)
* Aproximación multidimensional

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data_src/DelayedFlights.csv", nrows=400000)

In [5]:
df = df.sample(250000)

In [33]:
# Seleccionamos la variable de ArrDelay y quitamos NaN
x = df["ArrDelay"].dropna()

### Regla del IQR para detección de outliers

In [12]:
# Obtenemos los cuartiles de dicha variable y el IQR. Esto lo hacemos para la posterior definición de umbral superior
# y umbral inferior para la determinación de outliers
length = len(x)
Q1 = np.percentile(x,25)
Q3 = np.percentile(x,75)
IQR = Q3-Q1

In [13]:
umbral_superior = Q3 + 1.5*IQR
umbral_inferior = Q1 - 1.5*IQR

In [14]:
umbral_superior

127.5

In [16]:
umbral_inferior

-60.5

Con esto hemos obtenido que cualquier valor de 'x' que esté por encima de 127.5 será considerado un atípico (por arriba) de nuestra distribución. Y cualquier valor por debajo de -60.5 (en este caso tiempo negativo es que el avión llega antes de su hora, luego tiene sentido) será considerado un atípico (por debajo) de nuestra distribución.

In [22]:
# Para conocer el número total de valores de x que se considerarían outliers:
sum(x > umbral_superior)

17584

In [29]:
# Esto supone un % del total de valores en x
(sum(x > umbral_superior) / length)*100

# También se puede calcular com:
np.mean(x > umbral_superior)*100

7.0626415821859485

In [32]:
# Y para el caso de atípicos por debajo:
np.mean(x < umbral_inferior)*100

0.0012049547740308148

### Uso del paquete EllipticEnvelope de sklearn.covariance para estudiar varias variables a la vez

In [34]:
from sklearn.covariance import EllipticEnvelope

In [35]:
# Creamos el objeto 'outliers' de tipo EllipticEnvelope con parámetro contamination=.01
#
# Esto nos crea este objeto "transformador" (un modelo) que lo que hace es, con dicho valor de parámetro, seleccionar
# el 1% de datos que considere "más alejados" de nuestros datos más centrados

outliers = EllipticEnvelope(contamination=.01)

In [36]:
# Elegimos ahora un conjunto de variables cuantativas presentes en nuestro dataframe de las cuáles queremos analizar la
# presencia de outliers

var_ls = ["DepDelay","TaxiIn","TaxiOut","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]

El empleo de EllipticEnvelope para selección de outliers presume que las variables del análisis siguen distribuciones normales y por supuesto son cuantitativas. Esto realmente no lo sabemos en este caso nos vale como recursos didáctico.

In [51]:
# Al objeto EllipticEnvelope no puedo pasarle NaN con lo que debo eliminarlos.

# Asimismo el conjunto de valores tiene que pasarse al método .fit mejor si es np.array

x = np.array(df.loc[:,var_ls].dropna())
x

array([[23., 12., 15., ..., 19.,  0.,  2.],
       [13.,  3., 18., ...,  4.,  0.,  1.],
       [21.,  6.,  7., ...,  0.,  0.,  0.],
       ...,
       [38.,  5., 36., ..., 36.,  0., 23.],
       [59.,  5.,  5., ...,  0.,  0., 51.],
       [48.,  3., 12., ...,  2.,  0., 48.]])

In [49]:
# Entrenamos el objeto transformador (es decir, fiteamos)

outliers.fit(x)

EllipticEnvelope(contamination=0.01)

In [56]:
# Con el objeto ya entrenado podemos acudir al método .predict para determinar qué valores son considerados outliers:

pred_outliers = outliers.predict(x)
pred_outliers

array([1, 1, 1, ..., 1, 1, 1])

In [67]:
# Nos devuelve un array de 1 y -1 indicando con -1 que obs/registro es un outlier

elips_outliers = np.where(pred_outliers==-1)[0]
elips_outliers

array([    49,    271,    363, ..., 165214, 165308, 165331], dtype=int64)

In [78]:
# 'elips_outliers' es ahora un array con el valor de los indices de pred_outliers que es = -1 y que, a su vez, pues son
# los índices de x que son outliers :))

df.loc[:,var_ls].dropna().reset_index().loc[elips_outliers]

,index,DepDelay,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
49,331810,906.0,14.0,25.0,906.0,0.0,4.0,0.0,0.0
271,121416,420.0,6.0,7.0,0.0,0.0,418.0,0.0,40.0
363,178927,261.0,6.0,14.0,0.0,0.0,242.0,0.0,0.0
488,44926,234.0,3.0,25.0,0.0,0.0,230.0,0.0,0.0
489,69788,210.0,5.0,18.0,0.0,0.0,296.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
165040,96051,73.0,5.0,309.0,0.0,32.0,278.0,0.0,41.0
165090,59296,234.0,8.0,65.0,0.0,0.0,275.0,0.0,0.0
165214,172886,377.0,11.0,41.0,0.0,0.0,381.0,0.0,0.0
165308,332098,959.0,16.0,48.0,959.0,0.0,33.0,0.0,0.0


En total EllipticEnvelope ha detectado 1658 observaciones como outliers